In [1]:
import os, sys

import numpy as np
import pandas as pd
import random
import tensorflow as tf

module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
  sys.path.append(module_path)

from utils.utils import Create_Similarity_Model

import matplotlib.pyplot as plt
import matplotlib.font_manager as font_manager
from mpl_toolkits import mplot3d
plt.rc('mathtext', fontset="cm")
import matplotlib.ticker as mtick
import seaborn as sns
import matplotlib

sns.set_style('whitegrid')
sns.set_context("talk")

2022-09-15 13:47:17.455526: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-09-15 13:47:17.455542: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [2]:
n_nonsimilar = 2
n_similar = 1

In [10]:
possible_re_tau = np.linspace(start = 10, stop = 100, num = 10)
possible_he = np.linspace(start = 100, stop = 10000, num = 100)
possible_r = np.linspace(start = 0.1, stop = 0.2, num = 100)
Re_tau = []
He_ = []
R_ =[]
Y_ = []
U_ = []
dic_x = {}
dic_y = {}

for re_tau in possible_re_tau:
    for he in possible_he:
        #possible_y = np.linspace(start = re_tau / 8, stop = re_tau, num = 100)
        for r in possible_r:
            R_.append(r)
            #Y_.append(y_)
            He_.append(he)
            Re_tau.append(re_tau)
            u_ = ((re_tau / 2) * (1 - (r ** 2))) - ((1 - r) * (he / re_tau))
            U_.append(u_)

# fig = plt.figure(figsize = (10, 7))
# ax = plt.axes(projection ="3d")

# surf = ax.scatter(np.array(R_), np.array(He_) / (np.array(Re_tau) ** 2), np.array(U_) / np.array(Re_tau))
# plt.show()

c = list(zip(R_, He_, Re_tau, U_))
random.shuffle(c)
R_, He_, Re_tau, U_= zip(*c)

dic_x['r'] = R_
dic_x['He'] = He_
dic_x['Re_tau'] = Re_tau
dic_y['u+'] = U_


nonsimilar_keys = ['r', 'He']

xtrain_df = pd.DataFrame.from_dict(dic_x)

xtrain = np.log(xtrain_df.values)
ytrain = pd.DataFrame.from_dict(dic_y)
print(ytrain)

              u+
0       0.604959
1      -5.959229
2       9.917355
3      10.999770
4      22.923554
...          ...
99995  20.452066
99996  32.000000
99997 -95.811157
99998  -5.620248
99999  32.317355

[100000 rows x 1 columns]


In [5]:
loss = 'mean_squared_error'
optimizer = tf.keras.optimizers.Adam(learning_rate = 1e-2)
best_loss = 50.0
n_fits = 1                       # Number of model fits we will try to perform
n_epochs_initial = 10          # Number of epochs we will train inside the loop
n_epochs_final = 10         # Number of epochs we will train after convergence is established
successfull_tries = 0
models = []

model = Create_Similarity_Model(n_nonsimilar = n_nonsimilar, n_similar = n_similar)
#model.summary()
model.compile(loss = loss, optimizer = optimizer)
model.fit(xtrain, ytrain, epochs = n_epochs_initial, verbose = 1)

# while successfull_tries < n_fits:
#     model = Create_Similarity_Model(n_nonsimilar = n_nonsimilar, n_similar = n_similar)
#     model.compile(loss = loss, optimizer = optimizer)
#     model.fit(xtrain, ytrain, epochs = n_epochs_initial, verbose = 0)
#     current_loss = model.evaluate(xtrain, ytrain, verbose = 0)
#     print("current loss:" + str(current_loss))
#     if current_loss < 12000:
#         successfull_tries += 1
#         model.fit(xtrain, ytrain, epochs = n_epochs_final, verbose = 0)
#         good_loss = model.evaluate(xtrain, ytrain, verbose = 0)
#         models.append(model)
#         print("Iteration " + str(successfull_tries) + ": current loss = " + str(good_loss))
#         if good_loss < best_loss:
#             best_model = model
#             best_loss = good_loss

Epoch 1/10
1044/3125 [=========>....................] - ETA: 5s - loss: nan

KeyboardInterrupt: 

In [ ]:
good_loss = model.evaluate(xtrain, ytrain, verbose = 1)

In [6]:
j = 0

for model in models:

    for i in range(n_nonsimilar):
        similarity_exponents = model.get_layer('similarity_layer_' + str(i+1)).weights[0][0].numpy()
        print('Exponents multiplicating inside the function for model ' + str(j) + ' in ' + nonsimilar_keys[i] + ': ' + str(similarity_exponents))

    multiplication_exponents = model.get_layer('multiplication_layer').weights[0][0].numpy()
    print('Exponents multiplicating outside the function for model ' + str(j) + ': '  + str(multiplication_exponents))
    j += 1

for i in range(n_nonsimilar):
    similarity_exponents = best_model.get_layer('similarity_layer_' + str(i+1)).weights[0][0].numpy()
    print('Exponents multiplicating inside the function for  best model: ' + nonsimilar_keys[i] + ': ' + str(similarity_exponents))

multiplication_exponents = best_model.get_layer('multiplication_layer').weights[0][0].numpy()
print('Exponents multiplicating outside the function for best model: '  + str(multiplication_exponents))

NameError: name 'best_model' is not defined

In [8]:
for i in range(n_nonsimilar):
    similarity_exponents = model.get_layer('similarity_layer_' + str(i+1)).weights[0][0].numpy()
    print('Exponents multiplicating inside the function for model ' + str(j) + ' in ' + nonsimilar_keys[i] + ': ' + str(similarity_exponents))

multiplication_exponents = model.get_layer('multiplication_layer').weights[0][0].numpy()
print('Exponents multiplicating outside the function for model ' + str(j) + ': '  + str(multiplication_exponents))

Exponents multiplicating inside the function for model 0 in r: [0.28505814]
Exponents multiplicating inside the function for model 0 in He: [0.6912769]
Exponents multiplicating outside the function for model 0: [-1.6544552]
